In [1]:
# =========================================================
# 1. Imports
# =========================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt

In [2]:
# =========================================================
# 2. Load dataset
# =========================================================
df = pd.read_csv("data/online_shoppers_intention.csv")
print("Dataset loaded:", df.shape)

Dataset loaded: (12330, 18)


In [3]:
# =========================================================
# 3. Convert boolean variables to numeric
# =========================================================
df['Revenue'] = df['Revenue'].astype(int)
df['Weekend'] = df['Weekend'].astype(int)


# =========================================================
# 4. Group rare categories in numeric-categorical features
# =========================================================
numeric_categorical = ['OperatingSystems', 'Browser', 'Region', 'TrafficType']

def group_rare(series, threshold=50):
    freq = series.value_counts()
    return series.apply(lambda x: x if freq[x] > threshold else "Other")

for col in numeric_categorical:
    df[col] = df[col].astype(str)
    df[col] = group_rare(df[col])


# =========================================================
# 5. Define categorical & numerical sets
# =========================================================
categorical_nominal = ['Month', 'VisitorType'] + numeric_categorical
numerical_features_original = [
    'Administrative', 'Administrative_Duration',
    'Informational', 'Informational_Duration',
    'ProductRelated', 'ProductRelated_Duration',
    'BounceRates', 'ExitRates', 'PageValues',
    'SpecialDay'
]


# =========================================================
# 6. Train-test split (BEFORE encoding!)
# =========================================================
X = df.drop('Revenue', axis=1)
y = df['Revenue']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, " Test:", X_test.shape)


# =========================================================
# 7. Log-transform duration columns
# =========================================================
for col in ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']:
    X_train[col] = np.log1p(X_train[col])
    X_test[col] = np.log1p(X_test[col])


# =========================================================
# 8. Fit OneHotEncoder on TRAIN split
# =========================================================
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoder.fit(X_train[categorical_nominal])

# Transform
X_train_cat = encoder.transform(X_train[categorical_nominal])
X_test_cat  = encoder.transform(X_test[categorical_nominal])

X_train = X_train.drop(columns=categorical_nominal)
X_test = X_test.drop(columns=categorical_nominal)

# Get feature names (optional)
encoded_cols = encoder.get_feature_names_out(categorical_nominal)


# =========================================================
# Define numerical columns AUTOMATICALLY
# =========================================================
numerical_features = X_train.select_dtypes(include=[np.number]).columns

X_train_num = X_train[numerical_features].reset_index(drop=True)
X_test_num  = X_test[numerical_features].reset_index(drop=True)


X_train_full = np.hstack([X_train_num.values, X_train_cat])
X_test_full  = np.hstack([X_test_num.values,  X_test_cat])



# =========================================================
# 10. Scale numerical + encoded features
# =========================================================
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled  = scaler.transform(X_test_full)


# =========================================================
# 11. PCA (for visualization only)
# =========================================================
pca = PCA(n_components=2)
pca.fit(X_train_scaled)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# =========================================================
# 12. SMOTE (APPLY ONLY ON SCALED DATA, ONLY FOR TRAINING MODELS THAT NEED IT)
# =========================================================
# Example: only for tree models, not for LogisticRegression

smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("After SMOTE:", X_train_smote.shape)


Train: (9864, 17)  Test: (2466, 17)
After SMOTE: (16676, 55)


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: divide by zero encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: overflow encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encounter

# Perceptron

In [6]:
from sklearn.linear_model import Perceptron
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

model = Perceptron(max_iter=1000, eta0=1.0, random_state=42)
model.fit(X_train_smote, y_train_smote)

y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8008921330089214
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.81      0.87      2084
           1       0.42      0.75      0.54       382

    accuracy                           0.80      2466
   macro avg       0.68      0.78      0.71      2466
weighted avg       0.86      0.80      0.82      2466



/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


## Tunning hyperparameters

In [11]:
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline

param_grid = {
    'clf__alpha': [1e-6, 1e-5, 1e-4, 1e-3],
    'clf__eta0': [0.001, 0.01, 0.1, 1]
}

pipe = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('clf', Perceptron(max_iter=1000))
])

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1')
grid.fit(X_train_scaled, y_train)

print("Best parameters:", grid.best_params_)
print("Best F1 score:", grid.best_score_)


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validati

Best parameters: {'clf__alpha': 1e-06, 'clf__eta0': 0.01}
Best F1 score: 0.5099505017898427


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/pytho